In [ ]:
using StatsBase
using Distributions

In [8]:
using TensorFlow
using MLDataUtils

In [9]:
using DensityEstimationML
using Plots

In [10]:
immutable NeuralDensityEstimator
    sess::Session
    
    #Network nodes
    optimizer::Tensor
    t::Tensor
    pdf::Tensor
    cdf::Tensor
end

In [112]:
a = Vector{Int}(6)
a[:]=2
a

6-element Array{Int64,1}:
 2
 2
 2
 2
 2
 2

In [194]:
leaky_relu6(z) = 0.001z + nn.relu6(z)

function NeuralDensityEstimator(prob_layer_sizes)
    sess = Session(Graph())
    @tf begin
        t1 = placeholder(Float32, shape=[1, -1])
        t2 = placeholder(Float32, shape=[1, -1])
        countprob_between = placeholder(Float32, shape=[-1])
        
        zp1 = [t1]
        zp2 = [t2]
        layer_sizes= [1; prob_layer_sizes; 1]
        act_funs = Vector{Function}(length(layer_sizes)-1)
        act_funs[:] = nn.relu6
        act_funs[end] = nn.sigmoid
        for ii in 2:length(layer_sizes)
            below_size = layer_sizes[ii-1]
            above_size = layer_sizes[ii]
            
            act_fun = act_funs[ii-1]
            
            Wii = get_variable("W_$ii", [above_size, below_size], Float32)
            bii = get_variable("b_$ii", [above_size, 1], Float32)
            
            push!(zp1, act_fun(Wii*zp1[end] .+ bii))
            push!(zp2, act_fun(Wii*zp2[end] .+ bii))
        end
        
        cdf1 = zp1[end]
        cdf2 = zp2[end]
        #Assumes t2>t1
        cdf_between = cdf2 - cdf1
                
        
        losses = 0.5.*(cdf_between .- countprob_between).^2
        
        loss=reduce_mean(losses; axis=2)
        optimizer = train.minimize(train.AdamOptimizer(), loss)
        
        pdf1 = gradients(cdf1, t1)
    end
    
    run(sess, global_variables_initializer())
    
    NeuralDensityEstimator(sess, optimizer, t1, pdf1, cdf1)
end

NeuralDensityEstimator

In [195]:
function Distributions.cdf(est::NeuralDensityEstimator, t::Real)
    gr = est.sess.graph
    ts = reshape([t], (1,1))
    run(est.sess, est.cdf, Dict(est.t=>ts))[1]
end

function Distributions.pdf(est::NeuralDensityEstimator, t::Real)
    gr = est.sess.graph
    ts = reshape([t], (1,1))
    run(est.sess, est.pdf, Dict(est.t=>ts))[1]
end



In [196]:
"""
Returns 3 vectors.
A vector of start points \$t1\$
A vector of end points \$t2\$
A vector of the counts of elements \$ti\$ between them, such that \$\left{t_i \mid t1 \le ti < t2 \right}\$
"""
function get_cdf_training_pairs(observations)
    observations = sort(observations)
    
    t1s = Float32[]
    t2s = Float32[]
    probs = Float32[]
    for (ii, t1) in enumerate(observations)
        for jj in ii+1 : length(observations)
            t2 = observations[jj]
            count = jj - ii           
            push!(t1s, t1)
            push!(t2s, t2)
            push!(probs, count/length(observations))
        end
    end
    
    t1s, t2s, probs
end

get_cdf_training_pairs

In [197]:
function StatsBase.fit!(estimator::NeuralDensityEstimator, observations;
    epochs = 20,
    batch_size = 1024)
    gr = estimator.sess.graph
    
    for ii in 1:epochs
        batch_losses = Float32[]
        for (t1_o, t2_o, probs_o) = eachbatch(shuffleobs(get_cdf_training_pairs(observations)), batch_size)
            loss_o, _, = run(estimator.sess, 
                [gr["loss"], estimator.optimizer],
                Dict(gr["t1"]=>t1_o', gr["t2"]=>t2_o', gr["countprob_between"]=>probs_o))
            push!(batch_losses, loss_o[1])
        end
        epoch_loss = mean(batch_losses)
        println("Epoch $ii: loss: $(epoch_loss)")
    end
    estimator
end

In [205]:
est = NeuralDensityEstimator([32, 64])
data = GenerateDatasets.magdon_ismail_and_atiya();


2017-08-07 17:56:49.634704: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1030] Creating TensorFlow device (/gpu:0) -> (device: 0, name: GeForce GTX TITAN X, pci bus id: 0000:01:00.0)
Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] (::Base.##716#717)(::Float64, ::TensorFlow.Tensor{Float32}) at ./deprecated.jl:346
 [3] (::TensorFlow.###8#9#11{Base.##716#717})(::Array{Any,1}, ::Function, ::Float64, ::Vararg{Any,N} where N) at /home/uniwa/students2/students/20361362/linux/.julia/v0.6/TensorFlow/src/meta.jl:67
 [4] NeuralDensityEstimator(::Array{Int64,1}) at ./In[194]:35
 [5] include_string(::String, ::String) at ./loading.jl:515
 [6] include_string(::Module, ::String, ::String) at /home/uniwa/students2/students/20361362/linux/.julia/v0.6/Compat/src/Compat.jl:577
 [7] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/uniwa/students2/students/20361362/linux/.julia/v0.6/IJulia/src/execute_request.jl:154
 [8] eventloop(::ZMQ.Socket) at /home/uniwa/students2/stu

In [206]:
fit!(est, data; epochs=100)

Epoch 1: loss: 0.08006751


INFO: The specified values for size and/or count will result in 444 unused data points
INFO: The specified values for size and/or count will result in 444 unused data points


Epoch 2: loss: 0.046977956
Epoch 3: loss: 0.0085209375


INFO: The specified values for size and/or count will result in 444 unused data points
INFO: The specified values for size and/or count will result in 444 unused data points


Epoch 4: loss: 0.005304319
Epoch 5: loss: 0.003396785


...

INFO: The specified values for size and/or count will result in 444 unused data points
INFO: The specified values for size and/or count will result in 444 unused data points


Epoch 98: loss: 0.0018465866
Epoch 99: loss: 0.0018522037


INFO: The specified values for size and/or count will result in 444 unused data points
INFO: The specified values for size and/or count will result in 444 unused data points


Epoch 100: loss: 0.0018517583


NeuralDensityEstimator(Session(Ptr{Void} @0x00007f8510d5deb0), <Tensor Group:1 shape=unknown dtype=Any>, <Tensor t1:1 shape=(1, ?) dtype=Float32>, <Tensor gradients/MatMul_grad/MatMul_3:1 shape=(1, ?) dtype=Float32>, <Tensor Sigmoid:1 shape=(1, ?) dtype=Float32>)

In [207]:
histogram(data)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -20 
 
 
 0 
 
 
 20 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

In [208]:
X=-50:0.1:100

plot(X, pdf.(est, X))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -50 
 
 
 -25 
 
 
 0 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 100 
 
 
 0.00 
 
 
 0.01 
 
 
 0.02 
 
 
 0.03 
 
 
 0.04 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,349.903 39.7386,349.803 40.1071,349.702 40.4756,349.601 40.8441,349.5 41.2126,349.398 41.5811,349.296 41.9496,349.194 42.3181,349.091 42.6866,348.988 
 43.0551,348.885 43.4236,348.782 43.7921,348.678 44.1606,348.574 44.5291,348.469 44.8976,348.364 45.2661,348.259 45.6346,348.154 46.0031,348.048 46.3717,347.942 
 46.7402,347.836 47.1087,347.729 47.4772,347.622 47.8457,347.515 48.2142,347.407 48.5827,347.299 48.9512,347.191 49.3197,347.082 49.6882,346.974 50.0567,346.864 
 50.4252,346.755 50.7937,346.645 51.1622,346.535 51.5307,346.424 51.8992,346.314 52.2677,346.203 52.6362,346.091 53.0047,345.98 53.3732,345.867 53.7417,345.755 
 54.1102,345.642 54.4787,345.529 54.8472,345.416 55.2157,345.303 55.5843,345.189 55.9528,345.074 56.3213,344.96 56.6898,344.845 57.0583,344.73 57.4268,344.614 
 57.7953,344.498 58.1638,344.382 58.5323,344.266 58.9008,344.149 59.2693,344.032 59.6378,343.914 60.0063,343.796 60.3748,343.678 60.7433,343.56 61.1118,343.441 
 61.4803,343.322 61.8488,343.203 62.2173,343.083 62.5858,342.963 62.9543,342.843 63.3228,342.722 63.6913,342.601 64.0598,342.48 64.4283,342.359 64.7969,342.237 
 65.1654,342.114 65.5339,341.992 65.9024,341.869 66.2709,341.746 66.6394,341.622 67.0079,341.499 67.3764,341.374 67.7449,341.25 68.1134,341.125 68.4819,341 
 68.8504,340.875 69.2189,340.749 69.5874,340.623 69.9559,340.497 70.3244,340.37 70.6929,340.243 71.0614,340.116 71.4299,339.988 71.7984,339.86 72.1669,339.732 
 72.5354,339.604 72.9039,339.475 73.2724,339.346 73.6409,339.216 74.0094,339.087 74.378,338.956 74.7465,338.826 75.115,338.695 75.4835,338.564 75.852,338.433 
 76.2205,338.301 76.589,338.17 76.9575,338.037 77.326,337.905 77.6945,337.772 78.063,337.639 78.4315,337.505 78.8,337.372 79.1685,337.237 79.537,337.103 
 79.9055,336.968 80.274,336.833 80.6425,336.698 81.011,336.563 81.3795,336.427 81.748,336.291 82.1165,336.154 82.485,336.017 82.8535,335.88 83.222,335.743 
 83.5906,335.605 83.9591,335.467 84.3276,335.329 84.6961,335.19 85.0646,335.051 85.4331,334.912 85.8016,334.773 86.1701,334.633 86.5386,334.493 86.9071,334.353 
 87.2756,334.212 87.6441,334.071 88.0126,333.93 88.3811,333.789 88.7496,333.647 89.1181,333.505 89.4866,333.363 89.8551,333.22 90.2236,333.077 90.5921,332.934 
 90.9606,332.79 91.3291,332.647 91.6976,332.503 92.0661,332.358 92.4346,332.214 92.8031,332.069 93.1717,331.924 93.5402,331.779 93.9087,331.633 94.2772,331.487 
 94.6457,331.341 95.0142,331.194 95.3827,331.048 95.7512,330.901 96.1197,330.753 96.4882,330.606 96.8567,330.458 97.2252,330.31 97.5937,330.162 97.9622,330.013 
 98.3307,329.864 98.6992,329.715 99.0677,329.566 99.4362,329.416 99.8047,329.266 100.173,329.116 100.542,328.966 100.91,328.815 101.279,328.665 101.647,328.514 
 102.016,328.362 102.384,328.211 102.753,328.059 103.121,327.907 103.49,327.755 103.858,327.602 104.227,327.449 104.595,327.296 104.964,327.143 105.332,326.99 
 105.701,326.836 106.069,326.682 106.438,326.528 106.806,326.374 107.175,326.219 107.543,326.064 107.912,325.909 108.28,325.754 108.649,325.599 109.017,325.443 
 109.386,325.287 109.754,325.131 110.123,324.975 110.491,324.818 110.86,324.661 111.228,324.505 111.597,324.347 111.965,324.19 112.334,324.033 112.702,323.875 
 113.071,323.717 113.439,323.559 113.808,323.401 114.176,323.242 114.545,323.084 114.913,322.925 115.282,322.766 115.65,322.607 116.019,322.447 116.387,322.288 
 116.756,322.128 117.124,321.968 117.493,321.808 117.861,321.648 118.23,321.488 118.598,321.327 118.967,321.166 119.335,321.006 119.704,320.845 120.072,320.683 
 120.441,320.522 120.809,320.361 121.178,320.199 121.546,320.037 121.915,319.875 122.283,319.713 122.652,319.551 123.02,3

In [209]:
empirical_cdf(data, X) = [length(filter(i->i<x, data)) for x in X]./length(data)

empirical_cdf (generic function with 1 method)

In [210]:
plot(X, [cdf.(est, X), empirical_cdf(data, X)] , label=["Estimated" "Empirical"])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -50 
 
 
 -25 
 
 
 0 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 100 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.3701,349.154 39.7386,349.042 40.1071,348.929 40.4756,348.816 40.8441,348.702 41.2126,348.588 41.5811,348.474 41.9496,348.359 42.3181,348.244 42.6866,348.128 
 43.0551,348.012 43.4236,347.895 43.7921,347.778 44.1606,347.661 44.5291,347.543 44.8976,347.425 45.2661,347.306 45.6346,347.187 46.0031,347.067 46.3717,346.947 
 46.7402,346.826 47.1087,346.705 47.4772,346.584 47.8457,346.462 48.2142,346.339 48.5827,346.217 48.9512,346.093 49.3197,345.969 49.6882,345.845 50.0567,345.721 
 50.4252,345.595 50.7937,345.47 51.1622,345.344 51.5307,345.217 51.8992,345.09 52.2677,344.963 52.6362,344.835 53.0047,344.706 53.3732,344.577 53.7417,344.448 
 54.1102,344.318 54.4787,344.188 54.8472,344.057 55.2157,343.926 55.5843,343.794 55.9528,343.662 56.3213,343.529 56.6898,343.396 57.0583,343.262 57.4268,343.128 
 57.7953,342.993 58.1638,342.858 58.5323,342.722 58.9008,342.586 59.2693,342.45 59.6378,342.312 60.0063,342.175 60.3748,342.037 60.7433,341.898 61.1118,341.759 
 61.4803,341.619 61.8488,341.479 62.2173,341.338 62.5858,341.197 62.9543,341.056 63.3228,340.913 63.6913,340.771 64.0598,340.627 64.4283,340.484 64.7969,340.339 
 65.1654,340.195 65.5339,340.049 65.9024,339.904 66.2709,339.757 66.6394,339.61 67.0079,339.463 67.3764,339.315 67.7449,339.167 68.1134,339.018 68.4819,338.868 
 68.8504,338.718 69.2189,338.568 69.5874,338.417 69.9559,338.265 70.3244,338.113 70.6929,337.96 71.0614,337.807 71.4299,337.653 71.7984,337.499 72.1669,337.344 
 72.5354,337.189 72.9039,337.033 73.2724,336.877 73.6409,336.72 74.0094,336.562 74.378,336.404 74.7465,336.245 75.115,336.086 75.4835,335.926 75.852,335.766 
 76.2205,335.605 76.589,335.444 76.9575,335.282 77.326,335.119 77.6945,334.956 78.063,334.792 78.4315,334.628 78.8,334.463 79.1685,334.298 79.537,334.132 
 79.9055,333.965 80.274,333.798 80.6425,333.631 81.011,333.462 81.3795,333.294 81.748,333.124 82.1165,332.954 82.485,332.784 82.8535,332.613 83.222,332.441 
 83.5906,332.269 83.9591,332.096 84.3276,331.922 84.6961,331.748 85.0646,331.574 85.4331,331.399 85.8016,331.223 86.1701,331.046 86.5386,330.869 86.9071,330.692 
 87.2756,330.514 87.6441,330.335 88.0126,330.155 88.3811,329.976 88.7496,329.795 89.1181,329.614 89.4866,329.432 89.8551,329.25 90.2236,329.067 90.5921,328.883 
 90.9606,328.699 91.3291,328.514 91.6976,328.329 92.0661,328.143 92.4346,327.956 92.8031,327.769 93.1717,327.581 93.5402,327.393 93.9087,327.204 94.2772,327.014 
 94.6457,326.824 95.0142,326.633 95.3827,326.441 95.7512,326.249 96.1197,326.056 96.4882,325.863 96.8567,325.669 97.2252,325.474 97.5937,325.279 97.9622,325.083 
 98.3307,324.886 98.6992,324.689 99.0677,324.491 99.4362,324.293 99.8047,324.094 100.173,323.894 100.542,323.694 100.91,323.493 101.279,323.291 101.647,323.089 
 102.016,322.886 102.384,322.682 102.753,322.478 103.121,322.273 103.49,322.068 103.858,321.862 104.227,321.655 104.595,321.448 104.964,321.24 105.332,321.031 
 105.701,320.821 106.069,320.611 106.438,320.401 106.806,320.189 107.175,319.977 107.543,319.765 107.912,319.552 108.28,319.338 108.649,319.123 109.017,318.908 
 109.386,318.692 109.754,318.475 110.123,318.258 110.491,318.04 110.86,317.822 111.228,317.602 111.597,317.383 111.965,317.162 112.334,316.941 112.702,316.719 
 113.071,316.496 113.439,316.273 113.808,316.049 114.176,315.825 114.545,315.599 114.913,315.374 115.282,315.147 115.65,314.92 116.019,314.692 116.387,314.463 
 116.756,314.234 117.124,314.004 117.493,313.773 117.861,313.542 118.23,313.31 118.598,313.077 118.967,312.844 119.335,312.61 119.704,312.375 120.072,312.14 
 120.441,311.904 120.809,311.667 121.178,311.43 121.546,311.192 121.915,310.953 122.283,310.713 122.65